# 行政区划转行政区代码

In [ ]:
import pandas as pd
df_table=pd.read_csv("矢量数据/行政区划.csv",encoding='GB18030',converters = {'code':str})
# code=df_table[df_table["region"].str.contains("洛龙区")].iloc[0,0]
# print(code)

In [ ]:
locations="阜南县、颍上县"

loc_list=locations.split("、")
code_text=""
for loc in range(len(loc_list)):
    code=df_table[df_table["region"].str.contains(loc_list[loc])].iloc[0,0]
    code_text=code_text+code+";"
code_text=code_text.strip(";")
print(code_text)

# csv数据转ttl

1. 处理DisasterEventType

由于DisasterEventType的值是按label赋的，所以在实际进行关联的时候，要转化为相应的IRI后缀（全部小写，去掉空格和短横线）

2. 拼接实例字符串

根据实例关系拼接三元组字符串

## 读取csv

In [5]:
import pandas as pd
df=pd.read_excel("./disaster-data/data.xlsx",sheet_name="partofData2",dtype={"hasBeginning": "string","hasEnd": "string"})
df.head(3)

,中文名,发生时间,DisasterIdentifier,hasBeginning,hasEnd,administrativeDivision,hasDisasterEventType,hasDisasterEventType.1,DisasterEventType,description
0,1939年天津水灾,1939-08-20,19390820_120000,1939-08-20 00:00:00,1939-10-10 00:00:00,120000,Storm Flood,NaN,Storm Flood,1939年天津水灾是1939年8月至10月期间在中国天津发生的一次洪水灾害。这场灾害造成当时...
1,1963年海河特大洪水,1963-08-01,19630801_130000,1963-08-02 00:00:00,1963-08-09 00:00:00,130000,Storm Flood,NaN,Storm Flood,1963年海河特大洪水：1963年8月海河流域的暴雨造成的特大洪水是海河南系（南运河、子牙河...
2,1975河南驻马店特大暴雨,1975-08-08,19750808_411700,1975-08-08 00:00:00,1975-08-08 00:00:00,411700,Dam-break Flood,NaN,Dam-break Flood,1975年8月，在河南省驻马店等地区、1万多平方公里的土地上，共计60多个水库相继发生垮坝溃...


## 新建一个ttl文件并写入前缀

In [2]:
# 创建一个ttl文件，文件名为filename,并向文件写入prefix
def ttl_create(filename, prefix):
    relative_path = "./rdf/"  # 新创建的txt文件的存放路径
    full_path = relative_path + filename + '.ttl'  
    file = open(full_path, 'w')
    file.write(prefix)   #msg也就是下面的Hello world!
    file.close()

name="灾害事件实例"
prefix='''@prefix : <http://www.semanticweb.org/wen/ontologies/2022/8/FloodDisasterEventOntology#> .
@prefix dc: <http://purl.org/dc/elements/1.1/> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix time: <http://www.w3.org/2006/time#> .
@prefix vann: <http://purl.org/vocab/vann/> .
@prefix terms: <http://purl.org/dc/terms/> .
@prefix schema: <http://schema.org/> .
@base <http://www.semanticweb.org/wen/ontologies/2022/8/FloodDisasterEventOntology> .

'''

ttl_create(name, prefix)


## 创建日期实例


In [6]:
# 获取日期列表
begin_list=df['hasBeginning'].tolist()
end_list=df["hasEnd"].tolist()
date_list=list(set(begin_list+end_list))

# 创建日期实例
def transfer_dateIndividual(dateStr):
    Y=dateStr[0:4]
    M=int(dateStr[4:6])
    D=int(dateStr[6:8])

    text=f'''###  http://www.semanticweb.org/wen/ontologies/2022/8/FloodDisasterEventOntology#{dateStr}
<http://www.semanticweb.org/wen/ontologies/2022/8/FloodDisasterEventOntology#{dateStr}> 
rdf:type owl:NamedIndividual ,time:Instant ;
time:day {D} ;
time:month {M} ;
time:year {Y} .

'''
    return text


def create_dateIndividual(filename,date_list):
    with open("./rdf/灾害事件实例.ttl", "r",encoding="utf-8") as f:  #打开文本
        origin_file = f.read()   #读取文本
        f.close()

    file = open(filename, 'a')


    # 遍历datelist
    for i in range(len(date_list)):
        date_list[i]=date_list[i].replace("-","")
        date_list[i]=date_list[i][0:8]
        dateindividual="http://www.semanticweb.org/wen/ontologies/2022/8/FloodDisasterEventOntology#"+str(date_list[i]);
        # 先检查有没有已存在的
        if dateindividual in origin_file:
            break
        dateIndi_text=transfer_dateIndividual(str(date_list[i]))
        file.write(dateIndi_text)   #msg也就是下面的Hello world!
    
    file.close()

create_dateIndividual("./rdf/灾害事件实例.ttl",date_list)

## 处理DisasterEventType

In [7]:
def transferDET(eventType):
    DisasterEventType={
        "Dam-break Flood":"dambreakflood", 
        "Ice Flood":"iceflood", 
        "Lake Flood":"lakeflood", 
        "Snowmelt Flood":"snowmeltflood", 
        "Storm Flood":"stormflood", 
        "Storm Surge Flood":"stormsurgeflood", 
        "Torrential Flood":"torrentialflood", 
        "Urban  Waterlogging":"urbanwaterlogging", 
        "Waterlogging":"waterlogging"
    }
    return DisasterEventType[eventType]

transferDET("Storm Flood")

'stormflood'

## 创建事件实例


In [13]:
def create_eventIndividual(df):
    text=""
    for index, row in df.iterrows():
        # print(index,row['中文名'].strip(),row['DisasterIdentifier']) 
        identifier=row['DisasterIdentifier']
        beginning=row['hasBeginning']
        beginning=beginning.replace("-","")
        beginning=beginning[0:8]
        end=row['hasEnd']
        end=end.replace("-","")
        end=end[0:8]
        label=row['中文名'].strip()
        aDivision=str(row['administrativeDivision']).replace(';',',')
        eventType=row['DisasterEventType'].replace(' ','').replace(';',",:").replace('-','').lower()
        description=row['description']
        if not pd.isnull(description):
            description=description.replace('\n', '')

        event_text=f'''###  http://www.semanticweb.org/wen/ontologies/2022/8/FloodDisasterEventOntology#{identifier}
<http://www.semanticweb.org/wen/ontologies/2022/8/FloodDisasterEventOntology#{identifier}> 
rdf:type owl:NamedIndividual , :DisasterEvent ;
:hasDisasterEventType :{eventType} ;
:begin <http://www.semanticweb.org/wen/ontologies/2022/8/FloodDisasterEventOntology#{beginning}> ;
:end <http://www.semanticweb.org/wen/ontologies/2022/8/FloodDisasterEventOntology#{end}> ;
:DisasterIdentifier "{identifier}" ;
:administrativeDivision {aDivision} ;
:description "{description}"^^rdfs:Literal ;
rdfs:label "{label}"@zh.

'''
        text=text+event_text
    return text

text= create_eventIndividual(df)
file = open("./rdf/灾害事件实例.ttl", 'a',encoding="utf-8")
file.write(text)
file.close()
